# Building an Ontology

# WHAT

1. **What are the “things” (nouns) we care about?**
   * e.g. *Movie*, *Actor*, *Director*, *Character*, *Genre*

2. **What are their relationships to each other (verbs)?**
   * *directed by, acted in, plays, belongs to, has title, has release date*

3. **What data (literals) do we need?**
   * *title (string), releaseDate (date), birthDate (date), runtime (integer)*

4. **What constraints make sense?**
   * Every Movie must have at least one Director.
   * A Character must be played by exactly one Actor.

## Ontology concepts:

* **Classes** (`owl:Class`): types of things.
* **ObjectProperties** (`owl:ObjectProperty`): links between individuals.
* **DatatypeProperties** (`owl:DatatypeProperty`): links to literal values.
* **Restrictions** (`owl:Restriction`): e.g. cardinality or value constraints.
* **Inverse properties** (`owl:inverseOf`): auto-generate reverse links.
* **Rules** (in N3) using the `log:` vocabulary for simple inference.

# Our first lines of code  
_just a sample, so we know how to write things..._

This is N3 (Notation 3) format of specifying an ontology.  
  
* **`@prefix`** declares namespaces. These are just like `import` statements in Python:
    * you are creating a space for your code and
    * you are including a component or a module (or all the classes, properties and methods of the component into your code).
* **`a`** is shorthand for `rdf:type`.
* **`;`** chains multiple predicates on the same subject.
* **`.`** ends a statement.

```n3
@prefix :    <http://example.org/movies#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# A simple triple:
:Movie a owl:Class ;
       rdfs:label "Movie" .

# A property:
:directedBy a owl:ObjectProperty ;
    rdfs:domain :Movie ;
    rdfs:range :Director ;
    rdfs:label "directed by" .
```

# Define the core classes

```n3
:Movie    a owl:Class ; rdfs:label "Movie" .
:Actor    a owl:Class ; rdfs:label "Actor" .
:Director a owl:Class ; rdfs:label "Director" .
:Character a owl:Class ; rdfs:label "Character" .
:Genre    a owl:Class ; rdfs:label "Genre" .
```

# Define Key Properties  

```n3
:directedBy    a owl:ObjectProperty ;
    rdfs:domain :Movie ;
    rdfs:range  :Director ;
    rdfs:label  "directed by" .

:actedIn       a owl:ObjectProperty ;
    rdfs:domain :Actor ;
    rdfs:range  :Movie ;
    rdfs:label  "acted in" .

:title         a owl:DatatypeProperty ;
    rdfs:domain :Movie ;
    rdfs:range  xsd:string ;
    rdfs:label  "title" .

:releaseDate   a owl:DatatypeProperty ;
    rdfs:domain :Movie ;
    rdfs:range  xsd:date ;
    rdfs:label  "release date" .
```

Different from OOP/relational - note that we are trying to create a _vocabulary_ for our world, in this all the properties have unique names, because each concept means something very specific. This gives us a chance to build a reasoning engine down the road.

# Add Inverse Properties

```n3
:directs       a owl:ObjectProperty ;
    owl:inverseOf :directedBy ; #suggestions - active voice, present tense "directing"
    rdfs:domain   :Director ;
    rdfs:range    :Movie ;
    rdfs:label    "directs" .

:playsCharacter a owl:ObjectProperty ;
    owl:inverseOf :characterPlayedBy ;
    rdfs:domain   :Actor ;
    rdfs:range    :Character ;
    rdfs:label    "plays character" .
```

# Introduce Restrictions

## **Restrictions in N3**

In N3, we attach constraints to a class via an anonymous `owl:Restriction` node inside an `rdfs:subClassOf`.

```n3
:MyClass
  rdfs:subClassOf [
    a owl:Restriction ;
    owl:onProperty   :someProperty ;
    <restriction>    <value> 
  ] .
```

* `owl:onProperty` – the property being constrained.
* `<restriction>` – one of the restriction types below.
* `<value>` – a literal (for cardinalities) or a class (for value restrictions).

## Basic Restriction Types

| Restriction          | Purpose                              | Example                                          |
| -------------------- | ------------------------------------ | ------------------------------------------------ |
| `owl:minCardinality` | At least *n* values for the property | `owl:minCardinality "1"^^xsd:nonNegativeInteger` |
| `owl:maxCardinality` | At most *n* values                   | `owl:maxCardinality "3"^^xsd:nonNegativeInteger` |
| `owl:cardinality`    | Exactly *n* values                   | `owl:cardinality "1"^^xsd:nonNegativeInteger`    |
| `owl:someValuesFrom` | At least one value of a given class  | `owl:someValuesFrom :Director`                   |
| `owl:allValuesFrom`  | All values must be of a given class  | `owl:allValuesFrom :Genre`                       |


### **Typed Literals in N3**

A literal in N3 can carry an explicit datatype. The general form is:

```n3
"<lexical form>"^^<datatypeIRI>
```

* **`"<lexical form>"`**
  The string of characters representing the value (e.g. `"1"`, `"2025-07-07"`).
* **`^^`**
  Separator indicating the literal is typed.
* **`<datatypeIRI>`**
  The IRI of the datatype, usually via a prefix like `xsd:`.

By default, an untyped literal without `^^` is `xsd:string`.

#### Examples

| Literal                       | Meaning                                              |
| ----------------------------- | ---------------------------------------------------- |
| `"1"^^xsd:integer`            | The integer value 1                                  |
| `"1"^^xsd:nonNegativeInteger` | The non-negative integer value 1                     |
| `"3.14"^^xsd:decimal`         | The decimal value 3.14                               |
| `"true"^^xsd:boolean`         | The Boolean value true                               |
| `"2025-07-07"^^xsd:date`      | The date July 7, 2025                                |
| `"Movie"`                     | A plain string (equivalent to `"Movie"^^xsd:string`) |
| `"Film"@en`                   | A language-tagged string in English                  |


...so when we jot

```n3
owl:cardinality   "1"^^xsd:nonNegativeInteger
```

we are trying to specify a constraint that the property will have exactly one value (the number 1 indicates one value).

### Restrictions for our Movie Ontology

Use `owl:Restriction` to enforce domain rules:

```n3
# Every Movie must have at least one Director
:Movie rdfs:subClassOf [
    a owl:Restriction ;
    owl:onProperty :directedBy ;
    owl:minCardinality "1"^^xsd:nonNegativeInteger
] .

# A Character is played by exactly one Actor
:Character rdfs:subClassOf [
    a owl:Restriction ;
    owl:onProperty :characterPlayedBy ;
    owl:cardinality "1"^^xsd:nonNegativeInteger
] .
```

# Simple Reasoning Rules in N3

#### Reasoning Rules in N3

##### Declare the log namespace
  ```n3
  @prefix log: <http://www.w3.org/2000/10/swap/log#> .
  ```

#### Sidebar: SWAP = Semantic Web Application Platform
**The log namespace**  
Here _log_ is short for _logic_ not _logging_ - don't ask. 

* **IRI**: `http://www.w3.org/2000/10/swap/log#` ([w3.org][1]) - (Internationalized Resource Identifier is a uniform way to name or “identify” resources on the Web, unlike URL or URI, this allows for Unicode chars, everything in an ontology can have it's own unique IRI, we borrow some standard ones from the web, so that our language remains consistent across ontologies)

In our case for e.g., we use `www.example.com` to provide a domain (this is more of a convention than anything else).  
So `:Movie` has an IRI `<http://example.org/movies#Movie>` and `:Actor` has an IRI `<http://example.org/movies#Actor>` and so on...

* **Purpose**: defines built-in predicates and functions used by CWM for rule-based reasoning in N3, e.g.:
  * `log:implies` – logical implication between graphs
  * `log:conclusion` – derive all conclusions of a formula
  * `log:includes` / `log:notIncludes` – test formula containment
  * `log:uri`, `log:semantics`, etc.

##### Rule structure
  ```n3
  { <antecedent graph> } 
    log:implies 
  { <consequent graph> } .
  ```

##### Variables
  * Prefixed with `?`, e.g. `?actor`, `?movie`.

##### Graphs
  * Curly braces `{ … }` enclose triple patterns.
  * Antecedent is “if”; consequent is “then”.

##### Inference
  * A reasoner matches the antecedent patterns and asserts the consequent triples.
```n3
{ ?actor :actedIn ?movie }
  log:implies
{ ?movie :hasActor ?actor } .
```

#### Now let's write a rule for our Movies N3

```n3
@prefix log: <http://www.w3.org/2000/10/swap/log#> .

# If an Actor actsIn a Movie, then that Movie has that Actor
{ ?actor :actedIn ?movie . }
  log:implies
{ ?movie :hasActor ?actor . } .
```

# All in all...  
here's what we've built.

1. Namespace declarations (IRIs etc.)
2. Ontology header
3. Class definitions
4. Object and datatype properties (with domains, ranges, labels)
5. Inverse property declarations
6. SubClassOf + Restrictions
7. N3 inference rules

See [o01-Movies.n3](./o01-Movies.n3) for one that I created...